In [1]:
import gdown

url = "https://drive.google.com/uc?id=19CzXudqN58R3D-1G8KeFWk8UDQwlb8is"
output = "food-11.zip"
gdown.download(url, output)

Downloading...
From: https://drive.google.com/uc?id=19CzXudqN58R3D-1G8KeFWk8UDQwlb8is
To: C:\Users\user\pythonwork\food-11.zip
1.16GB [01:39, 11.7MB/s]


'food-11.zip'

In [2]:
import zipfile

with zipfile.ZipFile("food-11.zip") as zip_ref:
    zip_ref.extractall()

In [3]:
url2 = "https://drive.google.com/uc?id=1CShZHsO8oAZwxQkMe7jRtEgSNb2w_OZu"
output2 = "checkpoint.pth"
gdown.download(url2, output2)

Downloading...
From: https://drive.google.com/uc?id=1CShZHsO8oAZwxQkMe7jRtEgSNb2w_OZu
To: C:\Users\user\pythonwork\checkpoint.pth
170MB [00:15, 11.1MB/s] 


'checkpoint.pth'

In [4]:
import os
import sys
import argparse
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from skimage.segmentation import slic
from lime import lime_image
from pdb import set_trace

OSError: [WinError 126] 找不到指定的模組。

In [ ]:
args = {
      'ckptpath': './checkpoint.pth',
      'dataset_dir': './food-11/'
}
args = argparse.Namespace(**args)

In [ ]:
print(args)

In [ ]:
# 這是助教的示範 model，寫作業時一樣要換成自己的
class Classifier(nn.Module):
  def __init__(self):
    super(Classifier, self).__init__()

    def building_block(indim, outdim):
      return [
        nn.Conv2d(indim, outdim, 3, 1, 1),
        nn.BatchNorm2d(outdim),
        nn.ReLU(),
      ]
    def stack_blocks(indim, outdim, block_num):
      layers = building_block(indim, outdim)
      for i in range(block_num - 1):
        layers += building_block(outdim, outdim)
      layers.append(nn.MaxPool2d(2, 2, 0))
      return layers

    cnn_list = []
    cnn_list += stack_blocks(3, 128, 3)
    cnn_list += stack_blocks(128, 128, 3)
    cnn_list += stack_blocks(128, 256, 3)
    cnn_list += stack_blocks(256, 512, 1)
    cnn_list += stack_blocks(512, 512, 1)
    self.cnn = nn.Sequential( * cnn_list)

    dnn_list = [
      nn.Linear(512 * 4 * 4, 1024),
      nn.ReLU(),
      nn.Dropout(p = 0.3),
      nn.Linear(1024, 11),
    ]
    self.fc = nn.Sequential( * dnn_list)

  def forward(self, x):
    out = self.cnn(x)
    out = out.reshape(out.size()[0], -1)
    return self.fc(out)

In [ ]:
model = Classifier().cuda()
checkpoint = torch.load(args.ckptpath)
model.load_state_dict(checkpoint['model_state_dict'])
# 基本上出現 <All keys matched successfully> 就是有載入成功，但最好還是做一下 inference 確認 test accuracy 沒有錯。